In [ ]:
import os

import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    logging,
    pipeline,
)
from trl import DataCollatorForCompletionOnlyLM, SFTTrainer

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

## Dataset

In [ ]:
dataset = load_dataset("neuralwork/fashion-style-instruct")

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'completion', 'context'],
        num_rows: 3193
    })
})


In [ ]:
print(dataset["train"][0])

{'input': "I'm a tall, athletic man with broad shoulders and a narrow waist. I prefer sharp, tailored suits that highlight my V-shaped torso.", 'completion': 'Outfit Combination 1:\n- Top: Fitted white linen shirt\n- Bottom: Slim-fit beige chinos\n- Shoe: Brown leather loafers\n- Accessories: Brown woven belt, aviator sunglasses\n\nOutfit Combination 2:\n- Top: Light blue oxford button-down shirt\n- Bottom: Navy blue tailored trousers\n- Shoe: Tan leather brogues\n- Accessories: Navy blue patterned pocket square, silver wristwatch\n\nOutfit Combination 3:\n- Top: Light gray tailored blazer\n- Bottom: Dark wash denim jeans\n- Shoe: White canvas sneakers\n- Accessories: Black leather belt, silver pendant necklace\n\nOutfit Combination 4:\n- Top: Navy blue polo shirt\n- Bottom: Khaki shorts\n- Shoe: Brown leather sandals\n- Accessories: Navy blue baseball cap, woven brown bracelet\n\nOutfit Combination 5:\n- Top: Light pink dress shirt (with rolled-up sleeves)\n- Bottom: Charcoal gray dre

In [ ]:
def format_instruction(sample):
    s = f"""You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the event described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
{sample["input"]}
    
### Context:
{sample["context"]}
    
### Response:
{sample["completion"]}
"""
    return s

In [ ]:
sample = dataset["train"][0]
print(format_instruction(sample))

You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the event described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
I'm a tall, athletic man with broad shoulders and a narrow waist. I prefer sharp, tailored suits that highlight my V-shaped torso.
    
### Context:
I'm going to a nature retreat.
    
### Response:
Outfit Combination 1:
- Top: Fitted white linen shirt
- Bottom: Slim-fit beige chinos
- Shoe: Brown leather loafers
- Accessories: Brown woven belt, aviator sunglasses

Outfit Combination 2:
- Top: Light blue oxford button-down shirt
- Bottom: Navy blue tailored trousers
- Shoe: Tan leather brogues
- Accessories: Navy blue patterned pocket square, silver wristwatch

Outfit Combination 3:
- Top: Light gray tailored blazer
- Bottom: Dark wash denim jeans
- Shoe: White canvas sneakers
- Accessories: Black leather belt, silver pendant necklace

Ou

## Quantization

In [ ]:
# BitsAndBytesConfig to quantize the model int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

## Model

In [ ]:
model_id = "mistralai/Mistral-7B-v0.3"

In [ ]:
# load model
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map="auto"
)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


## Finetuning PEFT

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    r=32,  # rank
    lora_alpha=64,  # how much weight should be given to original weights
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 85,065,728 || all params: 7,333,089,280 || trainable%: 1.1600258056588124


## Training

In [ ]:
model_args = TrainingArguments(
    output_dir="./mistral-7b-style",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False,
)

## Fine tune only for response

In [ ]:
### Huggingface implementation is incorrect make your own with masked.

In [ ]:
# response_template = "### Response:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
# def format_instruction2(inp, cntx, comp):
#     s = f"""You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the event described in the context to generate 5 self-contained and complete outfit combinations.
# ### Input:
# {inp}
    
# ### Context:
# {cntx}
    
# ### Response:
# {comp}
# """
#     return s

In [ ]:
# def formatting_prompts_func_lst(example):
#     # print(example)
#     output_texts = []
#     for i in range(len(example["input"])):
#         text = format_instruction2(
#             example["input"][i], example["context"][i], example["completion"][i]
#         )
#         output_texts.append(text)
#     return output_texts

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    # packing=False,
    packing=True,
    # formatting_func=formatting_prompts_func_lst,
    formatting_func=format_instruction,
    # data_collator=collator,
    args=model_args,
)

Generating train split: 0 examples [00:00, ? examples/s]

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.822700
20,0.660000
30,0.615300
